In [1]:
import roslibpy
import time
import types
import functools

import opentnsim
import pandas as pd
import geopandas as gpd
import simpy.rt
import shapely.geometry


/Users/baart_f/.virtualenvs/opentnsim/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [2]:
berths = gpd.read_file('/Users/baart_f/data/ros/terminal_berths_schematic.geojson')
berths = berths.set_crs('EPSG:4326')
berths_rd = berths.to_crs('EPSG:28992')



In [3]:
class HasRos:
    def __init__(self, client, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client = client

class RosListener(HasRos):
    def __init__(self, topic_names: list, on_message, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.topic_names = topic_names
        self.topics = {}
        self.on_message = types.MethodType(on_message, self)
        self.callbacks = {}
        for topic_name in topic_names:
            topic_type = self.client.get_topic_type(topic_name)
            topic = roslibpy.Topic(
                self.client, 
                topic_name, 
                topic_type

            )
            self.topics[topic_name] = topic
        self.subscribe()
    def subscribe(self):
        for topic_name in self.topics:
            self.callbacks[topic_name] = functools.partial(self.on_message, topic_name=topic_name)
        for topic_name, topic in self.topics.items():
            callback = self.callbacks[topic_name]
            topic.subscribe(callback)

class RosPublisher(opentnsim.core.Identifiable, HasRos):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.publish_topics = {}
        topic_name = f"/opentnsim_waypoint_{self.name}/log"
        topic_type = 'std_msgs/String'
        self.publish_topics['str'] = roslibpy.Topic(
            self.client, 
            topic_name, 
            topic_type
        )
        topic_name = f"/opentnsim_waypoint_{self.name}/pos"
        topic_type = 'sensor_msgs/NavSatFix'
        self.publish_topics['pos'] = roslibpy.Topic(
            self.client, 
            topic_name, 
            topic_type
        )
    def publish(self, message: str):
        self.publish_topic.publish(roslibpy.Message({'data': message}))
    
class WaypointPublisher(RosPublisher):
    def __init__(self, *args, **kwargs):
        super().__init__(*args ,**kwargs)
    def publish_next_waypoint(self):
        # LOGIC TO DETERMINE NEXT WAYPOINT HERE
        self.publish_topics['pos'].publish(roslibpy.Message({"data": "message"}))

Ship = type(
    'Ship', 
    (
        opentnsim.core.SimpyObject, 
        opentnsim.core.Locatable,
        RosListener, 
        RosPublisher
    ), 
    {}
)

In [4]:
# Connect to the ros websocket bridge
client = roslibpy.Ros(host='localhost', port=9090)

def on_ready(*args, **kwargs):
    print('Connected', client.is_connected)
client.on_ready(on_ready)
client.run()

Connected True


In [5]:

env = simpy.rt.RealtimeEnvironment()


def update_location(ship, message, **kwargs):
    last_message = message
    ship.geometry = shapely.geometry.Point(message['longitude'], message['latitude'])

ship = Ship(
    env=env, 
    geometry=shapely.geometry.Point(4.37, 52.0), 
    client=client, 
    topic_names=['/RAS_TN_GR/geopos_est'], 
    on_message=update_location,
    name="ship"
)
env.run()


In [12]:
for i in range(3):
    time.sleep(1)
    print(ship.geometry.wkt)

POINT (4.371963585 52.00152903666667)
POINT (4.371968998333333 52.00153157166667)
POINT (4.3719742733333335 52.00153462)


In [7]:
class TrackTopics():
    def __init__(self, *args, **kwargs):
        super().__init__(*args ,**kwargs)

        self.last_geometry_per_topic = {}
        
Operator = type(
    'Operator', 
    (
        opentnsim.core.SimpyObject, 
        RosListener, 
        RosPublisher,
        TrackTopics
    ), 
    {}
)

In [8]:


def update_availability(operator, message, topic_name):
    geometry = shapely.geometry.Point(message['longitude'], message['latitude'])
    operator.last_geometry_per_topic[topic_name] = geometry

operator = Operator(
    env=env, 
    client=client, 
    topic_names=['/RAS_TN_GR/geopos_est', '/RAS_TN_OR/geopos_est'], 
    on_message=update_availability,
    name="operator"
)


In [9]:
occupied = None
if operator.last_geometry_per_topic:

    ship_geometries = gpd.GeoDataFrame(geometry=list(operator.last_geometry_per_topic.values()), crs='EPSG:4326')
    berth_geometries = berths.to_crs('EPSG:28992').geometry.buffer(10).to_crs('EPSG:4326')
    occupied = berth_geometries.intersects(ship_geometries.dissolve().iloc[0].geometry)

occupied

In [10]:
gpd.GeoDataFrame(geometry=list(operator.last_geometry_per_topic.values()), crs='EPSG:4326')

,geometry
